In [ ]:
# install botocore and boto3
# %pip install botocore boto3
# %pip install mypy-boto3-iam
# %pip install mypy-boto3-cloudformation
# %pip freeze > requirements.txt
# %pip install mypy-boto3-s3

In [ ]:
import boto3
import json

# Create IAM client
iam = boto3.client('iam')

# Create a role
role_name = 'my-test-role'
assume_role_policy_document = {
    'Version': '2012-10-17',
    'Statement': [
        {
            'Effect': 'Allow',
            'Principal': {
                'Service': 'ec2.amazonaws.com'
            },
            'Action': 'sts:AssumeRole'
        }
    ]
}

create_response = iam.create_role(
    RoleName=role_name,
    AssumeRolePolicyDocument=json.dumps(assume_role_policy_document)
)

print('Role ARN:', create_response['Role']['Arn'])

In [ ]:
delete_response = iam.delete_role(RoleName=role_name)
print('Role deleted, status:', delete_response['ResponseMetadata']['HTTPStatusCode'], 
      'and requestId: ', delete_response['ResponseMetadata']['RequestId'])

In [ ]:

from pathlib import Path
import json

print('Project Rootpath: ', rootpath.resolve())

trust_file_path=rootpath/Path('cicd-role/trust-relationship.json')
print('Trust File Path: ', trust_file_path.resolve())

with open(trust_file_path, 'r') as trust_file:
    trust_policy = json.load(trust_file)
    print(trust_policy)

# replace variables in trust policy


In [ ]:
from enum import Enum

class Environment(Enum):
    development = "dev"
    testplan = "tpe"
    experiment = "exp"
    production = "prd"

env_name = "development"

if isinstance(env_name, Environment):
    print(f"Environment: {env_name}")
else:
    print("Invalid Environment")

In [ ]:
from string import Template
from pathlib import Path
import json

rootpath = Path('../..')

def read_json_file_template(file_path: Path) -> Template:
    with open(file_path, 'r') as file:
        json_content=json.load(file)
        return Template(json.dumps(json_content))


In [ ]:
cdk_policy_path=rootpath/Path('cicd-role/policies/assume-cdk-policy.json')
cdk_policy=read_json_file_template(cdk_policy_path)
print("cdk policy template variables: ", cdk_policy.get_identifiers())

In [ ]:
cli_policy_path=rootpath/Path('cicd-role/policies/cli-policy.json')
cli_policy=read_json_file_template(cli_policy_path)
print("cli policy template variables: ", cli_policy.get_identifiers())

In [ ]:
inline_policies_dir = rootpath/Path('cicd-role/policies/inline')
for inline_policy_path in inline_policies_dir.rglob("*.json"):
    inline_policy_name=inline_policy_path.stem
    inline_policy_template=read_json_file_template(inline_policy_path)
    print(f"inline policy {inline_policy_name}, template variables: ", inline_policy_template.get_identifiers())
    # print("is template valid?", inline_policy_template.is_valid(), 'reason: ', [a for a in inline_policy_template.pattern.finditer(inline_policy_template.template)])
    print(inline_policy_template.safe_substitute({'aaws_principal_account': '123456789012'}))


In [ ]:
dist_dir = Path("dist")
print("dist dir: ", dist_dir.resolve()," isExists? ", dist_dir.exists())
text_file=dist_dir/"mytext.txt"
print("Text File Path:", text_file.resolve())
print([f.resolve() for f in dist_dir.rglob("*.txt")])
env_name="development"
env=Environment[env_name]
print(env_name, env, isinstance(env, Environment), isinstance(env_name, Environment))
# print(Environment[Environment[env]])
print("".join([s.capitalize() for s in "api-lambda-policy".split("-")]))

In [ ]:
import boto3

iam = boto3.client("iam")
def get_policy(policy_name: str):
    get_policy_response = iam.get_policy(
            PolicyArn=f"arn:aws:iam::471112824979:policy/{policy_name}")

    print(get_policy_response)
    return get_policy_response

In [ ]:
get_policy("CdkApiLambda")

get_policy("DummyCdkApiLambda")


In [ ]:
import enum

print(type(Environment))
print(isinstance(Environment, enum.EnumType))

In [ ]:
import boto3
from mypy_boto3_cloudformation import CloudFormationClient
import json

cft:CloudFormationClient = boto3.client("cloudformation")

def print_stack_result(stack_name:str):
    result = cft.describe_stacks(
            StackName=stack_name
        )

    print("Describe Stack: ", stack_name,
              "Status: ", result["ResponseMetadata"]["HTTPStatusCode"])
    print("describe output", json.dumps(result["Stacks"][0], indent=4, default=str))
    

In [ ]:
# try:
#     print_stack_result(f'CDKToolkit-prsfin-deva')
# except Exception as e:
#     print(e)
#     if "An error occurred (ValidationError) when calling the DescribeStacks" in f"{e}":
#         print("found validation error")

import re

e_typ="stack"
matched=re.match(r".+"+re.escape(e_typ)+r".+ does not exist.*", "botocore.exceptions.ClientError: An error occurred (ValidationError) when calling the DescribeStacks operation: Stack with id CDKToolkit-prsfin-development does not exist", re.IGNORECASE)
print(matched)

In [ ]:
import boto3
from mypy_boto3_cloudformation import CloudFormationClient
import json

cft: CloudFormationClient = boto3.client('cloudformation')

describe_response=cft.describe_stack_events(StackName="CDKToolkit-prsfin-testplan")

print(json.dumps(describe_response, indent=4, default=str))


In [ ]:
from datetime import datetime

event_time=describe_response["StackEvents"][0]["Timestamp"].astimezone()
print(event_time)
now=datetime.now().astimezone()
print(now)
print(datetime.now())

print(event_time>now)
print(event_time>=now)
print("---")
print(event_time<now)
print(event_time<=now)

In [ ]:
from pathlib import Path

policy_path = Path("../../cdk-roles/cfn-exec-role/policies/custom/storage-policy.json")

print(policy_path.resolve(), policy_path.exists())

policy_path.match("cfn-exec-role/**")

In [16]:
request_form="""# Request Form Production Environment

This form is intended to standardize deployment requests for production environments, ensuring clarity and completeness

### Deployment Type:
  _(Select one option by marking an "X" in the corresponding checkbox.)_  
  
- [x] **Release**  
- [ ] **Rollback**  

### Release Details:
  _(Provide version details for release or rollback)_  

- **Rollback Version to Deploy:** v0.1.0    <!-- Specify the version to rollback release deployment, e.g., v0.1.0. -->
- **New Version to Deploy:** v0.2.0    <!-- Specify the version for a release deployment, e.g., v0.2.0. -->
- **Existing Version Deployed:** v0.1.0  
- **Release Notes:**  
    _(Provide a link to release notes or specify 'Not Applicable' for rollbacks)_  


### Environment Details:
- **Environment Name:** Production Environment  

### Deployment Schedule:
  _(Use the format mm-dd-yyyy HH:MM:SS - 24-hour time. It should current CST time.)_  
  
- **Preferred Date and Time:** 03-15-2025 13:40:35  

### Deployment Reason:

#### Risk Level:  
  _(Provide level from one of the following)_
- Low
- Medium 
- High  

#### Reason/Justification:  
  _(Explain why the selected risk level is appropriate.)_  

#### Deploy / Rollback Explaination:  
  _(Provide a brief explanation, features, issue links, e.g., "Bug fixes and performance improvements" or "Rollback due to critical issue in v0.1.0.")_  

### Rollback Plan:  
  
#### Trigger Condition:  
  _(Specify the conditions under which a rollback would be initiated)_  

#### Rollback Reason:  
  _( Provide issues, impact analysis, etc.)_  


### Post Deployment Tasks:

#### Verification Steps:  
    _(List checks to confirm successful deployment or rollback. or link to tests steps if applicable)_  

#### Smoke and PPV Test Verification:  
  _(Provide list of verifications to be performed after release deployment / rollback)_  

- [ ] all public links are navigational before login  
- [ ] successful login to desktop chrome browser  
- [ ] successful login to android chrome browser  
- [ ] all links for expenses are navigational after login  
- [ ] all links for payment account are navigational after login  
- [ ] all links for settings are navigational after login  

#### Health Check Verification:  
  _(Provide list of all manual or automated verifications are included in health check. following some examples given.)_  

- [ ] cf health of site link. workflow job  
- [ ] site is launch in desktop chrome browser  
- [ ] site is launch in android chrome browser  
- [ ] rest api call are working in site  

### Additional Notes or Special Instructions:
_(Enter any additional notes, special considerations, or requirements, e.g., "Ensure zero downtime deployment.")_  
  
  

> [!IMPORTANT]
> If release Deployment fails, and roolback is required, create a rollback request form as sub issue of this.

> [!TIP]
> When verifications are performed, update the checkmark if passed. in case of failure, add details.
"""

In [19]:
from markdown_to_json import dictify
from typing import Dict


def parsed_body(requestform_body: str, header1_dummy: bool = False, header2_dummy: bool = False) -> Dict:
    """
    Args:
        requestform_body (str):  request form issue body
        header1_dummy and header2_dummy are used for internal to correct the body to parse in dictionary
    Returns:
        Dictionary of header3 as key and content as value, parsed header3 content value could be of any of these types; list, dict or str.
    """
    parsed_req_form = dictify(requestform_body)
    if not isinstance(parsed_req_form, Dict) and not header1_dummy:
        # add header1 and retry
        return parsed_body("# Dummy\n"+requestform_body, header1_dummy=True)

    incorrect_format_error_message = "request form is not in correct format. Please follow template `Request  Regression - Provision/Deprovision Test Plan Environment`"

    if not isinstance(parsed_req_form, Dict) or len(parsed_req_form) != 1:
        raise TypeError(incorrect_format_error_message)

    ignorek, header1_value = parsed_req_form.popitem()
    if not isinstance(header1_value, Dict) and not header2_dummy:
        # add header2 and retry
        header1_index = requestform_body.find(ignorek)
        header1_end_index = requestform_body.find(
            "\n", header1_index+len(ignorek))
        return parsed_body(requestform_body[:header1_end_index] + "\n## Dummy" + requestform_body[header1_end_index:], header2_dummy=True)

    if not isinstance(header1_value, Dict) or len(header1_value) != 1:
        raise TypeError(incorrect_format_error_message)

    ignorek, header2_value = header1_value.popitem()
    if not isinstance(header2_value, Dict):
        raise TypeError(incorrect_format_error_message)

    return header2_value


In [20]:
from markdown_to_json import dictify
import json

form_dict = parsed_body(request_form)

print(json.dumps(form_dict, indent=4))

{
    "Deployment Type:": "_(Select one option by marking an \"X\" in the corresponding checkbox.)_  \n\n['[x] **Release**  ', '[ ] **Rollback**  ']",
    "Release Details:": "_(Provide version details for release or rollback)_  \n\n['**Rollback Version to Deploy:** v0.1.0    <!-- Specify the version to rollback release deployment, e.g., v0.1.0. -->', '**New Version to Deploy:** v0.2.0    <!-- Specify the version for a release deployment, e.g., v0.2.0. -->', '**Existing Version Deployed:** v0.1.0  ', \"**Release Notes:**  \\n_(Provide a link to release notes or specify 'Not Applicable' for rollbacks)_  \"]",
    "Environment Details:": [
        "**Environment Name:** Production Environment  "
    ],
    "Deployment Schedule:": "_(Use the format mm-dd-yyyy HH:MM:SS - 24-hour time. It should current CST time.)_  \n\n['**Preferred Date and Time:** 03-15-2025 13:40:35  ']",
    "Deployment Reason:": {
        "Risk Level:  ": "_(Provide level from one of the following)_\n\n['Low', 'Medium